# 🛠️ Manual de Instalación y Configuración del Entorno
**Proyecto:** Sistema de Segmentación de Palmeras (Amazonía Peruana)  
**Autor:** Instituto de Investigaciones de la Amazonía Peruana  
**Fecha:** 23/12/2025  
**Versión de Python:** 3.11 (Crítico)

Este documento detalla los pasos exactos para replicar el entorno de desarrollo necesario para ejecutar el sistema de IA y procesamiento geoespacial.

## 1. Requisitos del Sistema

Para garantizar el funcionamiento de los modelos ONNX y el procesamiento de imágenes GeoTIFF pesadas:

| Componente | Requisito Mínimo | Requisito Recomendado |
| :--- | :--- | :--- |
| **Sistema Operativo** | Windows 10 (64-bit) | Windows 11 (64-bit) |
| **Python** | Versión 3.11.x | **Python 3.11.x** (Estrictamente necesario para GDAL 3.6.4) |
| **RAM** | 8 GB | **16 GB o superior** (Recomendado para tiles grandes) |
| **Disco** | 10 GB libres | 50 GB libres (Para datasets temporales) |
| **Dependencias C++** | VC++ 2015-2022 | Microsoft Visual C++ Redistributable (Necesario para GDAL) |

## 2. 📂 Estructura del Repositorio

Organización modular siguiendo buenas prácticas de ingeniería de software. Es importante verificar que la carpeta `installers` contenga el archivo `.whl` de GDAL, ya que es un requisito crítico para la instalación offline.

```text
Sistema_Segmentacion_Palmeras/
│
├── 📜 run.py                       # Punto de entrada de la aplicación
├── ⚙️ config.json                   # Ajuste de modelos, parámetros y gestión de RAM
├── 📦 requirements.txt             # Lista de librerías y dependencias
├── 📘 Manual_instalacion.ipynb     # Documentación y guía de despliegue
│
├── 📁 installers/                  # Binarios críticos pre-compilados
│   └── GDAL-3.6.4-cp311-win_amd64.whl  # ⚠️ Archivo requerido para instalación manual
│
├── 📁 app/                         # Código fuente de la GUI (PyQt5) y lógica de control
│   ├── ... (archivos de la interfaz)
│
├── 📁 scripts/                     # Scripts core de segmentación y conteo (Tiles, Watershed)
│   ├── ... (lógica de procesamiento)
│
├── 📁 models/                      # Modelos de IA entrenados (.onnx)
│   ├── ... (archivos de modelos)
│
└── 📁 output/                      # Carpeta de generación de resultados

## 3. Configuración del Entorno y Dependencias en VS Code

Siga estos pasos en orden estricto para configurar el entorno de desarrollo utilizando Visual Studio Code.

### Paso 1: Abrir el Proyecto
1. Abra Visual Studio Code.
2. Vaya a **Archivo > Abrir Carpeta...** (File > Open Folder).
3. Seleccione la carpeta raíz `Sistema_Segmentacion_Palmeras`.

### Paso 2: Creación del Entorno Virtual
Necesitamos crear un entorno aislado llamado `onnx_env_311` para instalar las librerías.

1. Abra una nueva terminal en VS Code: Menú **Terminal > New Terminal** (o atajo `Ctrl + ñ`).
2. Asegúrese de tener **Python 3.11** instalado en su sistema.
3. Ejecute el siguiente comando para crear el entorno:

```powershell
python -m venv onnx_env_311

## 4. Descarga y Distribución de Archivos Críticos

Debido al gran tamaño de los modelos de Inteligencia Artificial y la naturaleza del binario de GDAL, estos archivos no se incluyen directamente en el repositorio de código y deben descargarse e integrarse manualmente.

### 4.1. Acceder al Repositorio de Recursos
Ingrese al siguiente enlace de Google Drive para acceder a los archivos necesarios:
👉 [Google Drive - Modelos y Dependencias](https://drive.google.com/drive/folders/1z7f_rpuceeKP_9uWC7lZt9XGeK9llSU_)

### 4.2. Archivos Requeridos
Debe descargar **únicamente** los siguientes 3 archivos:

1.  **Instalador GDAL:** `GDAL-3.6.4-cp311-cp311-win_amd64.whl`
2.  **Modelo de Instancias:** `model_converted.onnx`
3.  **Modelo de Segmentación:** `deeplab_keras_model_palms_iaa_all_0.003_W.onnx`

### 4.3. Distribución en el Proyecto
Una vez completada la descarga, mueva cada archivo a su carpeta correspondiente dentro de la estructura `Sistema_Segmentacion_Palmeras`.

> ⚠️ **Advertencia:** No modifique los nombres de los archivos. El archivo de configuración (`config.json`) depende de estos nombres exactos para funcionar.

| Archivo Descargado | Carpeta de Destino |
| :--- | :--- |
| `GDAL-3.6.4...win_amd64.whl` | 📂 **`installers/`** |
| `model_converted.onnx` | 📂 **`models/`** |
| `deeplab_keras...0.003_W.onnx` | 📂 **`models/`** |

### 4.4. Verificación Final
Antes de proceder con la instalación de librerías, asegúrese de que su estructura de carpetas se vea así:

```text
Sistema_Segmentacion_Palmeras/
├── 📂 installers/
│   └── GDAL-3.6.4-cp311-cp311-win_amd64.whl  ✅
│
├── 📂 models/
│   ├── model_converted.onnx                  ✅
│   └── deeplab_keras_model...0.003_W.onnx    ✅
│
└── ... (resto de carpetas y archivos)

## 5. Instalación de Dependencias y Librerías

La instalación de librerías geoespaciales (como GDAL) y de Inteligencia Artificial (como ONNX Runtime) en Windows requiere un orden estricto para evitar errores de compilación de C++.

Implementaremos una estrategia de **Instalación en Dos Fases**: primero el núcleo binario crítico y luego el resto de dependencias estándar.

> **⚠️ VERIFICACIÓN PREVIA OBLIGATORIA:**
> Antes de ejecutar cualquier comando, mire su terminal en VS Code. Debe ver el indicador `(onnx_env_311)` en color verde o entre paréntesis al inicio de la línea.
> * **✅ Correcto:** `(onnx_env_311) PS C:\Users\...\Sistema_Segmentacion_Palmeras>`
> * **❌ Incorrecto:** `PS C:\Users\...\Sistema_Segmentacion_Palmeras>`
>
> *Si no aparece el entorno activado, regrese al paso anterior y actívelo.*

---

### 5.1. Fase 1: Instalación Manual del Núcleo (GDAL)

**¿Por qué hacemos esto manualmente?**
Si intenta instalar GDAL directamente desde internet (`pip install gdal`), Windows intentará compilar el código fuente. Esto suele fallar si no tiene las herramientas de compilación de C++ instaladas, generando errores críticos.

**Solución:**
Utilizamos un archivo **Wheel (.whl)** local que descargó en la sección anterior. Este archivo ya contiene los binarios pre-compilados listos para funcionar.

Ejecute el siguiente comando en la terminal para instalar GDAL desde su carpeta `installers`:

```powershell
pip install "installers/GDAL-3.6.4-cp311-cp311-win_amd64.whl"

### 5.2. Fase 2: Instalación de Dependencias del Proyecto (requirements.txt)

Una vez que el núcleo crítico (GDAL) está instalado correctamente, el sistema está listo para recibir el resto de las librerías.

El archivo `requirements.txt`. Le indica a Python que descargue todas las herramientas necesarias (PyQt5 para la interfaz, NumPy para cálculos, ONNX Runtime para la IA, etc.) en sus versiones exactas para garantizar la compatibilidad.

Ejecute el siguiente comando para instalar todo el paquete restante de una sola vez:

```powershell
pip install -r requirements.txt

### 5.3. Verificación Final

Para confirmar que el entorno está completamente configurado y listo para ejecutar la aplicación, validaremos que las librerías críticas están presentes.

Ejecute el siguiente comando en la terminal:

```powershell
pip list



## 6. Ejecución de la Aplicación

Una vez configurado el entorno, el sistema está listo para operar. 

### Ejecución desde Terminal (Recomendada)
Esta opción es la más estable y permite visualizar los registros (logs) de procesamiento en tiempo real dentro de la consola, lo cual es útil para depuración.

1. Asegúrese de estar en la carpeta raíz del proyecto `Sistema_Segmentacion_Palmeras`.
2. Verifique que el entorno `(onnx_env_311)` siga activo en su terminal.
3. Ejecute el comando de lanzamiento:

```powershell
python run.py